In [2]:
#Recurrent Neural Networks

In [3]:
# Part 1 - Data preprocessing

In [4]:
import pandas as pd
import numpy as np #need numpy to convert to arrays - not allowed to pass dataframe to NN
import matplotlib.pyplot as plt

In [5]:
# Import training set
dataset_train =pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:,1:2].values

In [6]:
training_set

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

In [7]:
# Feature scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [8]:
training_set_scaled.shape

(1258, 1)

In [9]:
# Create a data structure with N timesteps and 1 output
# N timesteps : Meaning at each time t, the LSTM will look at N timesteps before time t
# Based on N previous timesteps it will predict the next output at time t+1
# Trial to find best N

# Create 2 entities
# 1. X train : input to NN
# 2. Y train : output

# Here with N=60, X train will contain 60 stock prices before that financial day
# Y train will contain the stock price of the next financial day

In [10]:
# Initialize
X_train = []
Y_train = []

In [11]:
# Here start from 60 since N=60
# end is last index of our observation (size of training_set) - remember size +1 !
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i,0])
    Y_train.append(training_set_scaled[i,0])

# make list to array
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [12]:
X_train.shape

(1198, 60)

In [13]:
X_train

array([[0.08581368, 0.09701243, 0.09433366, ..., 0.07846566, 0.08034452,
        0.08497656],
       [0.09701243, 0.09433366, 0.09156187, ..., 0.08034452, 0.08497656,
        0.08627874],
       [0.09433366, 0.09156187, 0.07984225, ..., 0.08497656, 0.08627874,
        0.08471612],
       ...,
       [0.92106928, 0.92438053, 0.93048218, ..., 0.95475854, 0.95204256,
        0.95163331],
       [0.92438053, 0.93048218, 0.9299055 , ..., 0.95204256, 0.95163331,
        0.95725128],
       [0.93048218, 0.9299055 , 0.93113327, ..., 0.95163331, 0.95725128,
        0.93796041]])

In [14]:
Y_train

array([0.08627874, 0.08471612, 0.07454052, ..., 0.95725128, 0.93796041,
       0.93688146])

In [15]:
Y_train.shape

(1198,)

In [16]:
# Reshaping - adding some more dimensionality to previous data structure
# Now we add unit : number of predictors we can use to predict target
# Here only one indicator. can add more predictors/indicators if needed

# Do this so that the form is compatible for future feeding to RNN

In [17]:
# reshape to 3d dimension.
# first dim is batch size - total number of observations we have
# second dim is timestep
# third dim is number of indicator (here 1)
X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1],1))

In [18]:
X_train.shape

(1198, 60, 1)

In [20]:
# Part 2 - Building RNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout #to avoid overfitting

In [22]:
#Initializing the RNN
regressor = Sequential()

In [24]:
# Adding the first LSTM layer & some Dropout regularization
# inside add specify what kind of layer to add- here LSTM layer
# inside LSTM() we specify 3 arguments
# 1. number of units : number of memory units,i.e. neurons you want to have in this LSTM layer
# need higher number to properly capture trend
# 2. Return sequences : set to true because we're building a stacked LSTM which therefore will have several layers. if not stacked set to false.
# 3. input shape : shape of input of X_train in 3D, but only need to input the last 2 (timesteps & indicators)
# because the first one (observations) will be automatically taken into account.
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))

#Now adding regularization layer
# takes 1 input which is the regularization rate - rate of neurons you want to drop/ignore during training
regressor.add(Dropout(0.2))

In [25]:
#Adding second LSTM layer
regressor.add(LSTM(units=50,return_sequences=True)) #no need to specify input shape anymore for further layer
regressor.add(Dropout(0.2))

#Adding third LSTM layer
regressor.add(LSTM(units=50,return_sequences=True)) #no need to specify input shape anymore for further layer
regressor.add(Dropout(0.2))

#Adding fourth LSTM layer
#Last layer return_sequences false since no more layer sequence after that
regressor.add(LSTM(units=50,return_sequences=False)) #no need to specify input shape anymore for further layer
regressor.add(Dropout(0.2))

In [26]:
#Adding final layer/output layer
regressor.add(Dense(units=1))

In [27]:
# Compiling the LSTM Model
# usually optimizer for RNN is rmsprop or adam
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [28]:
# Fitting to training set
regressor.fit(X_train,Y_train, epochs=100, batch_size=30)

Epoch 1/100
1198/1198 [==============================] - 13s 11ms/step - loss: 0.0449
Epoch 2/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0072
Epoch 3/100
1198/1198 [==============================] - 9s 7ms/step - loss: 0.0056
Epoch 4/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0052
Epoch 5/100
1198/1198 [==============================] - 8s 6ms/step - loss: 0.0047
Epoch 6/100
1198/1198 [==============================] - 7s 5ms/step - loss: 0.0041
Epoch 7/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0054
Epoch 8/100
1198/1198 [==============================] - 9s 8ms/step - loss: 0.0043
Epoch 9/100
1198/1198 [==============================] - 10s 8ms/step - loss: 0.0044
Epoch 10/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0047
Epoch 11/100
1198/1198 [==============================] - 10s 9ms/step - loss: 0.0040
Epoch 12/100
1198/1198 [==============================] - 9s 7ms/step 

1198/1198 [==============================] - 8s 7ms/step - loss: 0.0015
Epoch 98/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0015
Epoch 99/100
1198/1198 [==============================] - 9s 7ms/step - loss: 0.0015
Epoch 100/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0013


In [29]:
# Part 3 - Making the prdictions and vizualizing
dataset_test =pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:,1:2].values

# Do prediction using our LSTM
# Now we need to combine train and test set because to predict the test set we need previous values
dataset_total = pd.concat([dataset_train['Open'], dataset_test['Open']], axis=0)
inputs = dataset_total[len(dataset_total)-len(dataset_test)-60:].values
inputs = inputs.reshape(-1,1)

# Scale inputs but not actual test values
inputs = sc.transform(inputs)

In [30]:
X_test=[]
for i in range(60, 80):
    X_test.append(inputs[i-60:i,0])

# make list to array
X_test = np.array(X_test)

X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))

In [31]:
predicted_stock_price = regressor.predict(X_test)

In [32]:
predicted_stock_price = sc.inverse_transform(predicted_stock_price)